In [1]:
#Importar Bibliotecas  
import requests
import pandas as pd
from datetime import datetime

In [2]:
#Função para obter os dados da API
def get_api_data(endpoint):
    url = f"https://fakestoreapi.com/{endpoint}"
    response = requests.get(url)
    return response.json()

In [3]:
#Obter todos os carrinhos de compras
carts = get_api_data("carts")

In [4]:
#Calcular o número total de carrinhos de compras
total_carts = len(carts)
print(f"Total de carrinhos: {total_carts}")

Total de carrinhos: 7


In [5]:
#Dicionário para armazenar informações dos usuários
user_data = {
    "id_usuario": [],
    "data_carrinho": [],
    "principal_categoria": []
}

In [6]:
#Contagem de registros
total_records = len(carts)

# Registro de progresso
processed_records = 0

# Iterar sobre os carrinhos de compras
for cart in carts:
    try:
        id_usuario = cart["userId"]
        cart_date = datetime.strptime(cart["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
        products = cart["products"]
        
        # Atualizar o registro de progresso
        processed_records += 1

        # Adicionar uma nova entrada para cada carrinho
        user_data["id_usuario"].append(id_usuario)
        user_data["data_carrinho"].append(cart_date)
        
        # Se não houver produtos no carrinho, adicionar "Nenhuma" na coluna "principal_categoria"
        if not products:
            user_data["principal_categoria"].append("Nenhuma")
        else:
            # Para carrinhos não vazios, determinar a categoria principal
            categories = []
            for product in products:
                product_id = product["productId"]
                # Obter detalhes do produto usando o endpoint /products
                product_details = get_api_data(f"products/{product_id}")
                if "category" in product_details:
                    categories.append(product_details["category"])
            
            if categories:
                most_common_category = max(set(categories), key=categories.count)
                user_data["principal_categoria"].append(most_common_category)
            else:
                user_data["principal_categoria"].append(None)
    
    except Exception as e:
        print(f"Erro ao processar o carrinho: {e}")

# Verificação de Integridade
for key, value in user_data.items():
    if not value:
        print(f"A coluna {key} está vazia ou incompleta.")

# Após a iteração, exibir o total de registros processados
print(f"Total de registros processados: {processed_records}")

Total de registros processados: 7


In [7]:
#Criar DataFrame com os dados dos usuários
user_df = pd.DataFrame(user_data)
display(user_df)

,id_usuario,data_carrinho,principal_categoria
0,1,2020-03-02,men's clothing
1,1,2020-01-02,men's clothing
2,2,2020-03-01,electronics
3,3,2020-01-01,men's clothing
4,3,2020-03-01,jewelery
5,4,2020-03-01,electronics
6,8,2020-03-01,women's clothing


In [8]:
#Salvar DataFrame em formato CSV
user_df.to_csv("user_data.csv", index=False)